<a href="https://colab.research.google.com/github/ThaoPham96/DeepARV/blob/main/DeepARV_Sim_PubChemAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install RDKit.
%%capture
!pip install rdkit-pypi
!pip install pubchempy

import tensorflow as tf
import pandas as pd
import pubchempy as pc
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools


In [ ]:
class DeepARV:

  def __init__(self, n_models=5):
    # Load all models
    self.model_trained = {}
    self.n_models = n_models
      # Load reference drugs
    print('\nLoading reference drugs')
    self.reference_drugs = np.load('/content/drive/MyDrive/ref_list.npy',
                                   allow_pickle=True)


  def load_models(self, ens):

    print('\nLoading pretrained models...\n')
    for i in tqdm(range(self.n_models)):
      print(str(i))
      self.model_trained[i] = tf.keras.models.load_model('/content/drive/MyDrive/liverpool_hiv/DeepARV_revised/v2_revised/'+ str(ens) + '/DeepARV_model'+str(i)+'.keras')
    print('\n')


  def drugname_to_fp(self, name):
    '''
    Input is a drug name, output is list of array of the presence of
    neighbour atoms [1] within fingerprint
    '''
    # get drug smiles
    drug = pc.get_compounds(name, 'name')
    drug_smiles = drug[0].isomeric_smiles

    # Convert SMILES to Mol
    drug_mol = Chem.MolFromSmiles(drug_smiles)

    # Molecular to Morgan fingerprint bits
    fp2 = AllChem.GetMorganFingerprintAsBitVect(drug_mol,2,nBits=1024)
    fp2_list = list(fp2.GetOnBits())
    return np.array((fp2_list))


  def create_feature_vector(self, drug_fp):
    '''
    Geneate feature vector for drug (structural similarity profile)
    by calculating Tanimoto score against the reference
    list
    '''
    def tanimoto(fp1, fp2):
      # This function computes Tanimoto score between 2 drugs
      intersect  = np.intersect1d(fp1, fp2)
      union = np.union1d(fp1, fp2)
      sim_score = round(len(intersect) / len(union),4)
      return sim_score

    feature_vector = []
    for i in self.reference_drugs:
      sim_score = tanimoto(drug_fp, i)
      feature_vector.append(sim_score)
    return feature_vector


  def ensemble_prediction(self, d1_fv, d2_fv):

      # Concat into single input
      X = np.hstack((d1_fv, d2_fv)).reshape(1,-1)
      ## dict containing all results
      self.all_results = {}
      ##dict containing results from single ensemble
      self.temp_results = []
      for i in range(5):
          #make prediction
          #iterate through each ensemble
          self.all_results[i] = self.model_trained[i].predict(X)
      all_results_values = np.array(list(self.all_results.values()))
      max_idx = all_results_values.mean(axis=0).argmax(axis=1)

      self.final_prediction = max_idx
      if self.final_prediction == 0:
        self.clinical_pred = 'Green: No clinically significant '\
             'interaction expected.'
      if self.final_prediction == 1:
        self.clinical_pred = 'Yellow: Potential interaction of weak '\
             'clinical relevance for which additional action/monitoring '\
             'or dosage adjustment is not required.'
      if self.final_prediction == 2:
        self.clinical_pred = 'Amber: Potential clinically relevant '\
             'interaction that can be managed by clinical monitoring, '\
             'alteration of drug dosage or timing of administration.'
      if self.final_prediction == 3:
        self.clinical_pred = 'Red: These drugs should not be '\
             'co-administered as they may cause a deleterious effect '\
             '(e.g., loss of efficacy or toxicity of the ARV drug '\
             'or coadministered drug)'
      return

  def predict(self, drugname_1, drugname_2):
    ''' Models final prediction with confidence score and clinical explanation '''
    # Convert drugs to FP
    d1_fp = self.drugname_to_fp(drugname_1)
    d2_fp = self.drugname_to_fp(drugname_2)

     # Convert each drug to feature vector
    d1_fv = self.create_feature_vector(d1_fp)
    d2_fv = self.create_feature_vector(d2_fp)

    # Prediction from all 5 models
    self.ensemble_prediction(d1_fv, d2_fv)



    #print('Model output: ' , self.final_prediction)
    print('Clinical prediction: ', self.clinical_pred)


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('Rilpivirine','Griseofulvin')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:01,  3.96it/s]

1


...

3


 80%|████████  | 4/5 [00:01<00:00,  3.12it/s]

4


100%|██████████| 5/5 [00:01<00:00,  3.13it/s]




1/1 [==============================] - 0s 90ms/step
Clinical prediction:  Yellow: Potential interaction of weak clinical relevance for which additional action/monitoring or dosage adjustment is not required.


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('efavirenz','flecainide')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:03,  1.00it/s]

1


...

3


 80%|████████  | 4/5 [00:02<00:00,  1.83it/s]

4


100%|██████████| 5/5 [00:02<00:00,  1.97it/s]




1/1 [==============================] - 0s 88ms/step
Clinical prediction:  Green: No clinically significant interaction expected.


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('efavirenz','Pentamidine')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:01,  3.82it/s]

1


...

3


 80%|████████  | 4/5 [00:00<00:00,  4.13it/s]

4


100%|██████████| 5/5 [00:01<00:00,  4.04it/s]




1/1 [==============================] - 0s 78ms/step
Clinical prediction:  Yellow: Potential interaction of weak clinical relevance for which additional action/monitoring or dosage adjustment is not required.


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('efavirenz','Bendroflumethiazide')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:00,  4.23it/s]

1


...

3


 80%|████████  | 4/5 [00:00<00:00,  4.19it/s]

4


100%|██████████| 5/5 [00:01<00:00,  4.15it/s]




1/1 [==============================] - 0s 88ms/step
Clinical prediction:  Green: No clinically significant interaction expected.


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('efavirenz','pantoprazole')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:01,  3.92it/s]

1


...

3


 80%|████████  | 4/5 [00:00<00:00,  4.14it/s]

4


100%|██████████| 5/5 [00:01<00:00,  4.11it/s]




1/1 [==============================] - 0s 80ms/step
Clinical prediction:  Red: These drugs should not be co-administered as they may cause a deleterious effect (e.g., loss of efficacy or toxicity of the ARV drug or coadministered drug)


In [ ]:
deeparv = DeepARV()
for i in range(5):
  deeparv.load_models(i)
  deeparv.predict('efavirenz','xipamide')


Loading reference drugs

Loading pretrained models...



  0%|          | 0/5 [00:00<?, ?it/s]

0


 20%|██        | 1/5 [00:00<00:02,  1.64it/s]

1


...

3


 80%|████████  | 4/5 [00:01<00:00,  3.68it/s]

4


100%|██████████| 5/5 [00:01<00:00,  3.62it/s]




1/1 [==============================] - 0s 86ms/step
Clinical prediction:  Green: No clinically significant interaction expected.
